# BMI565: Bioinformatics Programming & Scripting

#### (C) Michael Mooney (mooneymi@ohsu.edu)

## Week 11: Benchmarking and Optimizing Python Code

1. Benchmarking / Profiling in Python
2. Optimizing Code with SciPy (Weave)
3. Parallel Processing
    - `Multiprocessing` module
    - `pp` (Parallel Python) module
4. Final Exam Review

#### Requirements

- Python 2.7
- `time`, `timeit`, and `profile` modules
- `scipy` and `numpy` modules
- `multiprocessing` module
- Parallel Python, `pp`, module

** Note: Parallel Python can be installed with `pip install pp`.

## Benchmarking / Profiling

There are a number of ways to evaluate the performance of your Python code. Three useful modules are:

- `time`
- `timeit`
- `profile`

In [1]:
## Define a function that determines if a number is prime
def isprime(n):
    """
    Returns the number if it is prime, otherwise returns None.
    """
    assert n > 0, "Number must be greater than 0!"
    if n < 2: return None
    for i in range(2,n):
        if n % i == 0:
            return None
    return n

def get_primes(min, max):
    result = []
    possible_primes = range(min,max+1)
    for n in possible_primes:
        result.append(isprime(n))

    prime_nums = [n for n in result if n is not None]
    return prime_nums

In [2]:
## Binary search function
def bsearch(l, n):
    s = 0
    e = len(l) - 1
    while True:
        if s > e:
            return None
        mid = (s + e)/2
        if l[mid] < n:
            s = mid  + 1
        elif l[mid] > n:
            e = mid  - 1
        else:
            return mid

In [3]:
## Recursive binary search function
def rec_bsearch(l,n,s=0,e=None):
    if e is None: e = len(l) - 1
    if s > e:
        return None
    mid = (s + e)/2
    if n == l[mid]:
        return mid
    elif n < l[mid]:
        return rec_bsearch(l,n,s,mid-1)
    else:
        return rec_bsearch(l,n,mid+1,e)

### `time` module

In [4]:
import time

def search_time(fun, N, M):
    runtimes = []
    nums = range(M)
    start_time = time.time()
    for i in range(N):
        t0 = time.time()
        cmd = fun + "(nums, 3450)"
        idx = eval(cmd)
        runtimes.append(time.time() - t0)
    
    print "Total runtime: ", time.time() - start_time
    print "Mean runtime: ", sum(runtimes)/len(runtimes)
    return None

In [5]:
print "Binary Search:"
search_time("bsearch", 5000, 1000000)

Binary Search:
Total runtime:  0.0782740116119
Mean runtime:  1.53031349182e-05


### `timeit` module

In [6]:
import timeit

## Get the runtime of a Python statement
timeit.timeit("bsearch(nums, 3450)", setup="from __main__ import bsearch; nums = range(1000000)", number=5000)

0.0261228084564209

In [7]:
## Create a timer and run it multiple times
timer = timeit.Timer("bsearch(nums, 3450)", setup="from __main__ import bsearch; nums = range(1000000)")
timer.repeat(3, number=5000)

[0.025300979614257812, 0.020846128463745117, 0.02072000503540039]

### `profile` module

In [8]:
import profile
nums = range(10000000)
profile.run("rec_bsearch(nums, 3450)")

         23 function calls (5 primitive calls) in 0.001 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.000    0.000 :0(len)
        1    0.000    0.000    0.000    0.000 :0(setprofile)
     19/1    0.000    0.000    0.000    0.000 <ipython-input-3-3e47e1fd46da>:2(rec_bsearch)
        1    0.000    0.000    0.000    0.000 <string>:1(<module>)
        0    0.000             0.000          profile:0(profiler)
        1    0.000    0.000    0.001    0.001 profile:0(rec_bsearch(nums, 3450))




## Scipy and `weave`

`Weave` allows you to optimize your code by including C/C++ code within your Python program. The weave.inline() function will run C code and return the results to your Python program. The weave.blitz() function will compile NumPy expressions for faster execution.

In [9]:
## The code below was adapted from the SciPy website:
## http://docs.scipy.org/doc/scipy/reference/tutorial/weave.html

## C binary search 
from scipy import weave
import numpy as np

def c_int_bsearch(l, n):
    """
    Binary search written in C, using SciPy weave
    """
    
    ## C code for binary search of integers
    c_code = """int val, mid, s = 0;
        int e = l.length() - 1;
        PyObject *py_val;
        while(1)
        {
            if (s > e)
            {
                return_val =  -1;
                break;
            }
            mid =  (s + e) /2;
            val = py_to_int(PyList_GetItem(l, mid), "val");
            if (val < n)
                s = mid + 1;
            else if (val > n)
                e = mid - 1;
            else
            {
                return_val = mid;
                break;
            }
        }
    """
    
    return weave.inline(c_code, ['l','n'])

In [10]:
c_int_bsearch([1,2,3,4,5,6,7,8], 3)

2

In [11]:
## Use weave.blitz() to compile and run a NumPy expression
a = np.random.randint(0,101,(3,50))
print a
print 
np_expr = "a[0,:] = (a[0,:] + a[1,:] + a[2,:])/3"
weave.blitz(np_expr)
print a

[[ 81  30  51  78  85  68  14  35   3  99  85  43  93  68  30  33   7  22
   34  26  51  72  49  73  60  66  64  27  21  47   6  15  47  86   8  86
   71  58  11   1  33  63 100  31  71  66  23  49  90  74]
 [ 85  42  61  55  31  93  57  21  32  93  11  82  55  20  95  97  22  67
    7  55  35  45  59  47  61  87   7  47  83  49  38  41  58  62  18  24
   86  80  81  18 100  87  64  61  39  12  51  76  81  76]
 [ 15  85  18  94  62  65  91  37  55  50  35  78  69   7  31  23  94  90
   99  52  38  58  31  33  37  98   1  20 100  43  15  69  26  31  33  79
   85  13  36  82   2  73  89  15  83  69  70  37  95  87]]

[[ 60  52  43  75  59  75  54  31  30  80  43  67  72  31  52  51  41  59
   46  44  41  58  46  51  52  83  24  31  68  46  19  41  43  59  19  63
   80  50  42  33  45  74  84  35  64  49  48  54  88  79]
 [ 85  42  61  55  31  93  57  21  32  93  11  82  55  20  95  97  22  67
    7  55  35  45  59  47  61  87   7  47  83  49  38  41  58  62  18  24
   86  80  81  18 100 

## Parallel Processing

Parallel processing is a technique for improving the performance of a computational task, based on the idea that large problems can often be split into multiple smaller problems. These smaller problems can then be solved simultaneously (in parallel). Given the constraints of processor design and development, parallel computing (multi-processor machines) is now a common way to improve computational power.

There are numerous Python modules that allow you to take advantage of the computational power of multiple processors (the list below is not complete):

[https://wiki.python.org/moin/ParallelProcessing](https://wiki.python.org/moin/ParallelProcessing)

Keep in mind that there is always some amount of overhead cost due to splitting a large task into multiple smaller task and then gathering/compiling the individual results. The efficiency gains achieved with parallel processing will depend on the individual tasks being performed and the amount of communication (data transfer) required.

In [12]:
import multiprocessing as mp
import pp

In [13]:
## Find prime numbers serially
min_prime = 30000
max_prime = 50000

t0 = time.time()
prime_nums = get_primes(min_prime, max_prime)
t1 = time.time()

print "There are %d prime numbers between %d and %d." % (len(prime_nums), min_prime, max_prime)
print "Elapsed time:", t1 - t0

There are 1888 prime numbers between 30000 and 50000.
Elapsed time: 11.852011919


In [14]:
profile.run("get_primes(min_prime, max_prime)")

         60008 function calls in 11.998 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
    20001    0.036    0.000    0.036    0.000 :0(append)
    20002    3.823    0.000    3.823    0.000 :0(range)
        1    0.000    0.000    0.000    0.000 :0(setprofile)
        1    0.081    0.081   11.998   11.998 <ipython-input-1-4d94308a6ebf>:13(get_primes)
    20001    8.057    0.000   11.880    0.001 <ipython-input-1-4d94308a6ebf>:2(isprime)
        1    0.000    0.000   11.998   11.998 <string>:1(<module>)
        1    0.000    0.000   11.998   11.998 profile:0(get_primes(min_prime, max_prime))
        0    0.000             0.000          profile:0(profiler)




### `Multiprocessing` module

`Multiprocessing` is a module in Python's standard library that allows you to spawn multiple Python processes. It is an easy way to take advantage of multiple cores on a single machine.

[https://docs.python.org/2/library/multiprocessing.html](https://docs.python.org/2/library/multiprocessing.html)

In [15]:
## Get number of CPUs
mp.cpu_count()

8

In [16]:
## Find prime numbers using parallel processes
possible_primes = range(min_prime,max_prime+1)

t2 = time.time()
pool = mp.Pool(processes=3)
result2 = pool.map(isprime, possible_primes)
prime_nums2 = [n for n in result2 if n is not None]
t3 = time.time()

## Make sure to close the processes created by Pool
pool.close()

print "There are %d prime numbers between %d and %d." % (len(prime_nums2), min_prime, max_prime) 
print "Elapsed time:", t3 - t2

There are 1888 prime numbers between 30000 and 50000.
Elapsed time: 4.24339699745


### `pp` (Parallel Python) module

The Parallel Python module can be used to parallelize across multiple processors on a single machine, and also across multiple nodes of a computing cluster.

[http://www.parallelpython.com/](http://www.parallelpython.com/)

In [17]:
## Create pp job server
job_server = pp.Server(ncpus=3)
jobs = []

t4 = time.time()
## Submit jobs to pp server
for i in possible_primes:
    jobs.append(job_server.submit(isprime, (i,)))
## Wait for all jobs to finish
job_server.wait()
prime_nums3 = [job() for job in jobs if job() is not None]
t5 = time.time()

## Close the processes created by pp
job_server.destroy()

## Print results
print "There are %d prime numbers between %d and %d." % (len(prime_nums3), min_prime, max_prime) 
print "Elapsed time:", t5 - t4

There are 1888 prime numbers between 30000 and 50000.
Elapsed time: 7.1587061882


## References

- [https://wiki.python.org/moin/ParallelProcessing](https://wiki.python.org/moin/ParallelProcessing)
- [http://docs.scipy.org/doc/scipy-0.14.0/reference/tutorial/weave.html](http://docs.scipy.org/doc/scipy-0.14.0/reference/tutorial/weave.html)
- [https://docs.python.org/2/library/time.html](https://docs.python.org/2/library/time.html)
- [https://docs.python.org/2/library/timeit.html](https://docs.python.org/2/library/timeit.html)
- [https://docs.python.org/2/library/profile.html](https://docs.python.org/2/library/profile.html)

#### Last Updated: 23-Sep-2016